In [87]:
# https://github.com/EN10/KerasMNIST/blob/master/cnnPredict.py
import requests
import os

url = 'https://github.com/EN10/KerasMNIST/blob/master/cnn.h5?raw=true'
if not os.path.exists('model.h5'):
    r = requests.get(url)
    with open("model.h5", 'wb') as f:
        f.write(r.content)
img_url = 'https://github.com/EN10/KerasMNIST/blob/master/test3.png?raw=true'
if not os.path.exists('test_3.png'):
    r = requests.get(img_url)
    with open("test_3.png", 'wb') as f:
        f.write(r.content)

In [88]:
from keras.models import load_model
model = load_model('model.h5')

/Users/kawasakitoshiya/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/keras/models.py:291: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [89]:
from scipy.misc import imread, imresize
import numpy as np
x = imread('test_3.png',mode='L')
x = np.invert(x)
#make it the right size
x = imresize(x,(28,28))
#convert to a 4D tensor to feed into our model
x = x.reshape(1,28,28,1)
x = x.astype('float32')
x /= 255

out = model.predict(x)
print(np.argmax(out))

3


In [90]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [91]:
# http://d.hatena.ne.jp/natsutan/20170219/1487515186
from keras import backend as K
get_1st_layer_output = K.function([model.layers[0].input],
                                  [model.layers[5].output])
layer_output = get_1st_layer_output([x,])
print(layer_output[0].shape)
print(layer_output[0])

(1, 128)
[[ 0.          0.          2.95871282  0.          0.42135167  0.42394322
   4.52540684  3.74002266  0.          0.          0.          0.          0.
   0.          0.          4.09314537  0.47893596  0.          3.43837309
   5.90915871  0.          0.          5.62233591  0.          2.01469922
   3.46936321  0.          0.          0.          0.82178652  3.98092747
   0.          2.93021107  4.86831665  0.          2.46259332  0.
   3.47192812  0.          5.48088789  0.          0.          0.          0.
   0.          0.          3.58125329  1.33336794  1.03526926  0.          0.
   0.          5.31381464  0.          0.          3.59103918  4.01049995
   0.          1.67447734  0.          0.          2.7813344   0.          0.
   0.          0.          1.76818252  5.47540236  4.27703953  3.01535988
   0.          0.          0.59004122  0.          0.          3.61456203
   0.          1.32712781  3.05374813  4.02648544  0.          0.
   4.1422348   0.          0.

In [92]:
# sklearn
from sklearn import datasets, model_selection, svm, metrics
mnist = datasets.fetch_mldata('MNIST original', data_home='data/src/download/')
mnist_data = mnist.data / 255
mnist_label = mnist.target

In [93]:
train_size = 10000
test_size = 2000
data_train, data_test, label_train, label_test = model_selection.train_test_split(
    mnist_data, mnist_label, test_size=test_size, train_size=train_size
)

In [94]:
num_images_train = _data_train.shape[0]
data_train = _data_train.reshape((num_images_train, 28, 28, 1))
num_images_test = _data_test.shape[0]
data_test = _data_test.reshape((num_images_test, 28, 28, 1))

In [95]:
print(data_train.shape)
print(data_test.shape)

(10000, 28, 28, 1)
(2000, 28, 28, 1)


In [96]:

data_train_feature = get_1st_layer_output([data_train,])[0]
data_test_feature = get_1st_layer_output([data_test,])[0]

In [ ]:
# Feature extract: cnn, clasification: svm
clf = svm.LinearSVC()
clf.fit(data_train_feature, label_train)
pre = clf.predict(data_test_feature)

In [98]:
ac_score = metrics.accuracy_score(label_test, pre)
print(ac_score)

0.098


In [99]:
# only CNN
result = model.predict([data_test, ])
pre = [np.argmax(_) for _ in result]

In [100]:
ac_score = metrics.accuracy_score(label_test, pre)
print(ac_score)

0.097
